# Import libraries

In [ ]:
!pip install transformers==4.32.0 accelerate tiktoken einops transformers_stream_generator==0.0.4 scipy torchvision pillow tensorboard matplotlib

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(1)

# Instantiate model

In [ ]:
# Instantiate Qwen-VL-Chat model. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## US-Politics

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_Qwen-VL-Chat.csv')
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_InternLM-XComposer.csv')
constraint22_dataset_uspolitics_test_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_llava-v1.5-13b.csv')
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_BLIP-2.csv')

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_llava = constraint22_dataset_uspolitics_test_captioned_llava.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.dropna().reset_index(drop=True)

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
# Use Qwen-VL-Chat for the inference. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
def get_prediction(image, prompt):
    query = tokenizer.from_list_format([
        {'image': image},
        {'text': prompt},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)
    return response

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x in {'hero', 'villain', 'victim', 'other'}:
        return x
    elif 'hero' in x and 'villain' not in x and 'victim' not in x and 'other' not in x and 'not hero' not in x and 'not a hero' not in x:
        return 'hero'
    elif 'villain' in x and 'hero' not in x and 'victim' not in x and 'other' not in x and 'not villain' not in x and 'not a villain' not in x:
        return 'villain'
    elif 'victim' in x and 'villain' not in x and 'hero' not in x and 'other' not in x and 'not victim' not in x and 'not a victim' not in x:
        return 'victim'
    elif 'other' in x and 'villain' not in x and 'victim' not in x and 'hero' not in x and 'not other' not in x and 'not an other' not in x:
        return 'other'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
uspolitics_test_grounding_absent_prompt_vanilla = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_vanilla_images = uspolitics_test_grounding_absent_prompt_vanilla['image'].values
uspolitics_test_grounding_absent_prompt_vanilla_entities = uspolitics_test_grounding_absent_prompt_vanilla['entity'].values
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(uspolitics_test_grounding_absent_prompt_vanilla_images, uspolitics_test_grounding_absent_prompt_vanilla_entities)]
uspolitics_test_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[VANILLA]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    742
other      180
hero        72
Name: prediction, dtype: int64
1
0.22585979758275163
              precision    recall  f1-score   support

        hero       0.51      0.15      0.23       246
     villain       0.29      0.85      0.43       250
      victim       0.00      0.00      0.00       249
       other       0.29      0.21      0.24       250

    accuracy                           0.30       995
   macro avg       0.27      0.30      0.23       995
weighted avg       0.27      0.30      0.23       995



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_OCR = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_OCR_images = uspolitics_test_grounding_absent_prompt_with_OCR['image'].values
uspolitics_test_grounding_absent_prompt_with_OCR_entities = uspolitics_test_grounding_absent_prompt_with_OCR['entity'].values
uspolitics_test_grounding_absent_prompt_with_OCR_texts = uspolitics_test_grounding_absent_prompt_with_OCR['OCR'].values
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(uspolitics_test_grounding_absent_prompt_with_OCR_images, uspolitics_test_grounding_absent_prompt_with_OCR_entities, uspolitics_test_grounding_absent_prompt_with_OCR_texts)]
uspolitics_test_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[OCR]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    508
other      210
hero       199
victim      27
Name: prediction, dtype: int64
51
0.3058384579162102
              precision    recall  f1-score   support

        hero       0.41      0.35      0.38       246
     villain       0.33      0.68      0.44       250
      victim       0.57      0.08      0.15       249
       other       0.27      0.24      0.26       250

    accuracy                           0.34       995
   macro avg       0.39      0.34      0.31       995
weighted avg       0.39      0.34      0.31       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    639
other      238
hero        85
victim       6
Name: prediction, dtype: int64
27
0.26022584208553895
              precision    recall  f1-score   support

        hero       0.55      0.21      0.30       246
     villain       0.31      0.79      0.44       250
      victim       0.45      0.02      0.04       249
       other       0.25      0.25      0.25       250

    accuracy                           0.32       995
   macro avg       0.39      0.32      0.26       995
weighted avg       0.39      0.32      0.26       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    625
other      250
hero        86
victim       4
Name: prediction, dtype: int64
30
0.24677507147552935
              precision    recall  f1-score   support

        hero       0.49      0.19      0.28       246
     villain       0.31      0.78      0.45       250
      victim       0.33      0.01      0.02       249
       other       0.24      0.25      0.24       250

    accuracy                           0.31       995
   macro avg       0.34      0.31      0.25       995
weighted avg       0.34      0.31      0.25       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_llava = constraint22_dataset_uspolitics_test_captioned_llava.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_llava_images = uspolitics_test_grounding_absent_prompt_with_caption_llava['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_entities = uspolitics_test_grounding_absent_prompt_with_caption_llava['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_captions = uspolitics_test_grounding_absent_prompt_with_caption_llava['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_llava_images, uspolitics_test_grounding_absent_prompt_with_caption_llava_entities, uspolitics_test_grounding_absent_prompt_with_caption_llava_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    478
other      303
hero        92
victim      14
Name: prediction, dtype: int64
108
0.2794524894508179
              precision    recall  f1-score   support

        hero       0.44      0.22      0.29       246
     villain       0.34      0.68      0.45       250
      victim       0.44      0.08      0.13       249
       other       0.22      0.28      0.25       250

    accuracy                           0.32       995
   macro avg       0.36      0.31      0.28       995
weighted avg       0.36      0.32      0.28       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

villain    743
other      196
hero        53
Name: prediction, dtype: int64
3
0.22015809619096827
              precision    recall  f1-score   support

        hero       0.60      0.13      0.21       246
     villain       0.28      0.84      0.42       250
      victim       0.00      0.00      0.00       249
       other       0.27      0.22      0.24       250

    accuracy                           0.30       995
   macro avg       0.29      0.30      0.22       995
weighted avg       0.29      0.30      0.22       995

